In [21]:
import scipy.misc
from numpy import genfromtxt
from PIL import Image, ImageDraw, ImageFont
import torch
import os
import imageio

In [3]:
# If you have a video put 1. If you have frames put 2.
# Based on your choice, update the below cells accordingly
video_frames = 1

# If you would like to visualize the flow frames. This WILL DELETE ALL THE FLOWFRAMES GENERATED TO SAVE STORAGE.
visualize = False 

# Video Input

if video_frames == 1:
    video_name = 'test2.mp4' # If you have a video you want to run inference on. Please include .mp4 or whatever extension the video has.
    video_local_gdrive = 2 # If you want to upload a video from your local drive, choose 1. If from your google drive, choose 2. If some other option, go to section "Upload Video".
  

# Frames Input

if video_frames == 2:
    frames_zip_name = "2 - Arabic.zip" # If you have the frames, enter its zip file here. For ex: "3 - Video.zip"
    frames_directory = '../gdrive/My Drive/Hajj Videos/Frames/' # Where the frames are located. If its in gdrive: '../gdrive/My Drive/Location of Zip/' Change the Location of Zip to wherever yours is stored.

# Skip / Average Options

no_frames_skip = 1 # How many frames you want skipped. For eg if its 2 then from frames 1,2,3,4,5,6,7 we take frames 1,4,7. Leave at None to not skip frames.

# Only one can work at a time
no_average_frames = None # How many frames you want to average. Leave at None if you don't want to avg.
running_average = False # If you want to visually see running average


if video_frames == 1:
    def mkdir_ifnotexists(dir):
        if os.path.exists(dir):
            return
        os.mkdir(dir)

if video_frames == 1:
    vid_file = video_name
    frame_pth = './frames'
    mkdir_ifnotexists(frame_pth)
    cmd = "ffmpeg -i %s -start_number 0 -vsync 0 %s/frame_%%06d.png" % (
                vid_file,
                frame_pth,
            )
    os.system(cmd)

if video_frames == 2:

    file_dir = "./frames/"
    for count, filename in enumerate(sorted(os.listdir(file_dir))):
        # print(filename)
        if filename[-11:] == "_UTC+01.jpg":
            src = file_dir + filename
            dst = file_dir + str(count).zfill(6) + '.png'
            os.rename(src, dst)

if no_frames_skip != None:
    directory = "./frames"

    temp_skip = no_frames_skip

    for i, file in enumerate(sorted(os.listdir(directory))):
        if (temp_skip != 0) and (file[-4:] == ".png"):
            os.remove(directory + '/' + file)
            temp_skip = temp_skip - 1
            continue
        temp_skip = no_frames_skip

if no_frames_skip != None:
    file_dir = "./frames/"
    for count, filename in enumerate(sorted(os.listdir(file_dir))):
        # print(filename)
        if filename[-4:] == ".png":
            src = file_dir + filename
            dst = file_dir + str(count).zfill(6) + '.png'
            os.rename(src, dst)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [4]:
dir = "./frames"
img = Image.open("./frames/000000.png")
# New Sizes
x = (img.size[0] // 64) * 64
y = (img.size[1] // 64) * 64
for frame in sorted(os.listdir(dir)):
    if frame[-3:] == 'png':

    # Open image and resize
        image = Image.open(dir + "/" + frame)
        new_image = image.resize((x, y))

        # Remove image and write new one
        os.remove(dir + "/" + frame)
        new_image.save(dir + "/" + frame)

f = open("./frames/test1.txt", "w")

i = 0
for frame in sorted(os.listdir("./frames")):
    if frame[-3:] == 'png':
        if i != 0:
            f.write("\n")        
        f.write(frame[:-4])
        i += 1

f.close()

In [2]:
import os
os.chdir('./NWPU-Crowd-Sample-Code/')
# !python test.py

In [58]:
!python test.py

modelPath:  ./model_out.pth
/home/omarmukhtar/anaconda3/envs/ogenv02Kelvin/lib/python3.7/site-packages/torch/_tensor.py:490: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
/home/omarmukhtar/anaconda3/envs/ogenv02Kelvin/lib/python3.7/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [14]:
!mkdir '../FinalFrames'

In [52]:
import pandas as pd
outResDIr = './Final_Results'
OriginalframesList = sorted(os.listdir('../frames'))
FinalResultOutList = sorted(os.listdir(outResDIr))

df_Counts = pd.read_csv('./Final_Results/All_Counts.txt', sep=' ', header=None)
countsList = df_Counts[:][1].values

for file_index in range(0,len(FinalResultOutList)-1):
    csvOutput = genfromtxt(outResDIr + '/' + FinalResultOutList[file_index+1], delimiter=',')
    im = imageio.imwrite('outfile.jpg', csvOutput)

    #blending imdages
    convert_imgs=[]
    convert_imgs.append(Image.open('outfile.jpg'))
    convert_imgs.append(Image.open('../frames/' + OriginalframesList[file_index]))
    blended_img = Image.blend(convert_imgs[0].convert("RGB"), convert_imgs[1].convert("RGB"), alpha=.20)
    # blended_img.save('.'+'/'+'blendedImOut.jpg')

    if len(countsList) > file_index:
        draw = ImageDraw.Draw(blended_img)

        # Choose a font and set the font size
        font = ImageFont.truetype('arial.ttf', 36)

        # Set the text color and draw the text
        text_color = (255, 255, 255)  # White
        position = (10, 10)  # Top left corner of the text
        draw.text(position, 'COUNT: {}'.format(countsList[file_index]), font=font, fill=text_color)

        # Save the modified image
    blended_img.save('../FinalFrames/'+ OriginalframesList[file_index] + '_OUT' + '.jpg')

Lossy conversion from float64 to uint8. Range [0.0, 1.693552017211914]. Convert image to uint8 prior to saving to suppress this warning.


(1280, 704) (1280, 704)


Lossy conversion from float64 to uint8. Range [0.0, 1.693552017211914]. Convert image to uint8 prior to saving to suppress this warning.


(1280, 704) (1280, 704)


Lossy conversion from float64 to uint8. Range [0.0, 1.5868185758590698]. Convert image to uint8 prior to saving to suppress this warning.


...

(1280, 704) (1280, 704)


Lossy conversion from float64 to uint8. Range [0.0, 1.4210439920425415]. Convert image to uint8 prior to saving to suppress this warning.


(1280, 704) (1280, 704)


Lossy conversion from float64 to uint8. Range [0.0, 1.3677177429199219]. Convert image to uint8 prior to saving to suppress this warning.


(1280, 704) (1280, 704)


Video

In [57]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

image_folder='../FinalFrames'
output_video_name = '../Hajj_Data_1.avi'
images = [img for img in sorted_alphanumeric(os.listdir(image_folder)) 
			if img.endswith(".jpg") or
				img.endswith(".jpeg") or
				img.endswith(".png")] 
frame = cv2.imread(os.path.join(image_folder, images[1])) 

# setting the frame width, height width 
# the width, height of first image 
height, width, layers = frame.shape 

video = cv2.VideoWriter(output_video_name, 0, 1, (width, height)) 

# Appending the images to the video one by one 
for image in images: 
    video.write(cv2.imread(os.path.join(image_folder, image))) 

# Deallocating memories taken for window creation 
cv2.destroyAllWindows() 
video.release() # releasing the video generated 